In [2]:
import os
from pathlib import Path
import joblib

CACHE_DIR = Path("cache")
CACHE_DIR.mkdir(exist_ok=True)
VECTORS_PATH = CACHE_DIR / "dense_vectors.pkl"
FILENAMES_PATH = CACHE_DIR / "filenames.pkl"
VECTORIZER_PATH = CACHE_DIR / "vectorizer.pkl"
LSH_CACHE_PATH = CACHE_DIR / "lsh_index.pkl"

def save_to_cache(dense_vectors, filenames, vectorizer):
    joblib.dump(dense_vectors, VECTORS_PATH)
    joblib.dump(filenames, FILENAMES_PATH)
    joblib.dump(vectorizer, VECTORIZER_PATH)

def load_from_cache():
    dense_vectors = joblib.load(VECTORS_PATH)
    filenames = joblib.load(FILENAMES_PATH)
    vectorizer = joblib.load(VECTORIZER_PATH)
    return dense_vectors, filenames, vectorizer

def is_cache_available():
    return VECTORS_PATH.exists() and FILENAMES_PATH.exists() and VECTORIZER_PATH.exists()


In [3]:
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

def load_license_texts_parallel(data_dir, max_workers=16):
    data_dir = Path(data_dir)
    files = list(data_dir.rglob("*.txt"))
    
    texts = []
    filenames = []

    def read_file(file_path):
        try:
            text = file_path.read_text(encoding="utf-8")
            return str(file_path.relative_to(data_dir)), text
        except Exception as e:
            return None

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        for result in tqdm(executor.map(read_file, files), total=len(files), desc="Reading files"):
            if result:
                rel_path, text = result
                filenames.append(rel_path)
                texts.append(text)

    return filenames, texts

In [4]:
import time
from pathlib import Path
from sentence_transformers import SentenceTransformer

project_root = Path("..").resolve()
output_base = project_root / "Combined-Licenses"

start_time = time.time()

if is_cache_available():
    print("[CACHE HIT] Loading Dense vectors...")
    dense_vectors, filenames, vectorizer = load_from_cache()
else:
    print("[CACHE MISS] Processing license texts...")
    t0 = time.time()
    filenames, texts = load_license_texts_parallel(output_base)
    sample_texts = texts[:10000]
    sample_filenames = filenames[:10000]
    
    model = SentenceTransformer("all-MiniLM-L6-v2")
    print(" Creating vectors on 10,000 samples only...")
    dense_vectors = model.encode(sample_texts, show_progress_bar=True)
    print("Vectors created for 10,000 samples.")
    save_to_cache(dense_vectors, sample_filenames, model)
    print(f"--- TF-IDF built in {time.time() - t0:.2f} sec")

[CACHE MISS] Processing license texts...


Reading files: 100%|██████████████████| 162835/162835 [00:21<00:00, 7570.79it/s]


 Creating vectors on 10,000 samples only...


Batches:   0%|          | 0/313 [00:00<?, ?it/s]

Vectors created for 10,000 samples.
--- TF-IDF built in 291.19 sec


In [7]:
# dense_vectors.shape
# len(sample_filenames)
# len(filenames)

162835

In [8]:
import sys
import time
import joblib
from tqdm import tqdm
from pathlib import Path

# Setup paths and import
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))
from LSH import LSH

start_time = time.time()

if LSH_CACHE_PATH.exists():
    print("[CACHE HIT] Loading existing LSH index...")
    lsh = joblib.load(LSH_CACHE_PATH)
else:
    print("[CACHE MISS] Building LSH index...")
    hash_size = 32
    input_dim = dense_vectors.shape[1]
    num_tables = 30
    lsh = LSH(hash_size, input_dim, num_tables)

    BATCH_SIZE = 500
    num_vectors = dense_vectors.shape[0]
    total_added = 0

    for start in tqdm(range(0, num_vectors, BATCH_SIZE), desc=" Adding vectors to LSH"):
        end = min(start + BATCH_SIZE, num_vectors)
        dense_batch = dense_vectors[start:end]
        for i, vector in enumerate(dense_batch):
            lsh.add(vector, sample_filenames[start + i])
            total_added += 1
            if total_added % 10000 == 0:
                print(f"[DEBUG] Added {total_added} vectors...")

    print("Saving LSH index to cache...")
    LSH_CACHE_PATH.parent.mkdir(parents=True, exist_ok=True)
    joblib.dump(lsh, LSH_CACHE_PATH)

print(f"Done! Total time: {time.time() - start_time:.2f} sec")


[CACHE MISS] Building LSH index...


🔗 Adding vectors to LSH:   0%|                          | 0/20 [00:00<?, ?it/s]

[DEBUG] Collision in table 0, bucket 10010110... → 2 items
[DEBUG] Collision in table 1, bucket 00100100... → 2 items
[DEBUG] Collision in table 2, bucket 10100000... → 2 items
[DEBUG] Collision in table 3, bucket 00000100... → 2 items
[DEBUG] Collision in table 4, bucket 00110000... → 2 items
[DEBUG] Collision in table 5, bucket 00011000... → 2 items
[DEBUG] Collision in table 6, bucket 00001011... → 2 items
[DEBUG] Collision in table 7, bucket 00000010... → 2 items
[DEBUG] Collision in table 8, bucket 10000000... → 2 items
[DEBUG] Collision in table 9, bucket 10100110... → 2 items
[DEBUG] Collision in table 10, bucket 00010000... → 2 items
[DEBUG] Collision in table 11, bucket 00000001... → 2 items
[DEBUG] Collision in table 12, bucket 00001100... → 2 items
[DEBUG] Collision in table 13, bucket 01000001... → 2 items
[DEBUG] Collision in table 14, bucket 00000000... → 2 items
[DEBUG] Collision in table 15, bucket 01010000... → 2 items
[DEBUG] Collision in table 16, bucket 00100000... 

🔗 Adding vectors to LSH:   5%|▉                 | 1/20 [00:00<00:04,  3.99it/s]

[DEBUG] Collision in table 0, bucket 00010011... → 2 items
[DEBUG] Collision in table 1, bucket 00000010... → 2 items
[DEBUG] Collision in table 2, bucket 10110001... → 2 items
[DEBUG] Collision in table 3, bucket 00111000... → 2 items
[DEBUG] Collision in table 4, bucket 00100000... → 2 items
[DEBUG] Collision in table 5, bucket 11000001... → 2 items
[DEBUG] Collision in table 6, bucket 00000000... → 2 items
[DEBUG] Collision in table 7, bucket 00000000... → 2 items
[DEBUG] Collision in table 8, bucket 10000000... → 2 items
[DEBUG] Collision in table 9, bucket 11100110... → 2 items
[DEBUG] Collision in table 10, bucket 01011011... → 2 items
[DEBUG] Collision in table 11, bucket 00000101... → 2 items
[DEBUG] Collision in table 12, bucket 10001001... → 2 items
[DEBUG] Collision in table 13, bucket 01000000... → 2 items
[DEBUG] Collision in table 14, bucket 00100001... → 2 items
[DEBUG] Collision in table 15, bucket 01010100... → 2 items
[DEBUG] Collision in table 16, bucket 00000000... 

...

🔗 Adding vectors to LSH:  95%|████████████████▏| 19/20 [00:04<00:00,  3.72it/s]

[DEBUG] Collision in table 0, bucket 10000000... → 2 items
[DEBUG] Collision in table 1, bucket 10000010... → 2 items
[DEBUG] Collision in table 2, bucket 00010010... → 2 items
[DEBUG] Collision in table 3, bucket 11010000... → 2 items
[DEBUG] Collision in table 4, bucket 00100100... → 2 items
[DEBUG] Collision in table 5, bucket 00011110... → 2 items
[DEBUG] Collision in table 6, bucket 01001000... → 2 items
[DEBUG] Collision in table 7, bucket 10000110... → 2 items
[DEBUG] Collision in table 8, bucket 10000010... → 2 items
[DEBUG] Collision in table 9, bucket 11100010... → 2 items
[DEBUG] Collision in table 10, bucket 01110110... → 2 items
[DEBUG] Collision in table 11, bucket 10000010... → 2 items
[DEBUG] Collision in table 12, bucket 01100100... → 2 items
[DEBUG] Collision in table 13, bucket 00000000... → 2 items
[DEBUG] Collision in table 14, bucket 00101000... → 2 items
[DEBUG] Collision in table 15, bucket 10110000... → 2 items
[DEBUG] Collision in table 16, bucket 00110000... 

🔗 Adding vectors to LSH: 100%|█████████████████| 20/20 [00:05<00:00,  3.88it/s]

[DEBUG] Collision in table 0, bucket 10010111... → 4 items
[DEBUG] Collision in table 1, bucket 00000000... → 4 items
[DEBUG] Collision in table 2, bucket 01110000... → 4 items
[DEBUG] Collision in table 4, bucket 00000000... → 4 items
[DEBUG] Collision in table 5, bucket 01101101... → 4 items
[DEBUG] Collision in table 6, bucket 00001010... → 4 items
[DEBUG] Collision in table 7, bucket 00000001... → 4 items
[DEBUG] Collision in table 8, bucket 10000000... → 4 items
[DEBUG] Collision in table 9, bucket 00100111... → 4 items
[DEBUG] Collision in table 10, bucket 00111110... → 4 items
[DEBUG] Collision in table 11, bucket 01000001... → 4 items
[DEBUG] Collision in table 12, bucket 00000101... → 4 items
[DEBUG] Collision in table 13, bucket 01101001... → 4 items
[DEBUG] Collision in table 14, bucket 01000000... → 5 items
[DEBUG] Collision in table 15, bucket 10010100... → 4 items
[DEBUG] Collision in table 16, bucket 00110001... → 4 items
[DEBUG] Collision in table 18, bucket 00001110...

Done! Total time: 5.62 sec


162835

In [11]:
import numpy as np
import time
from tqdm import tqdm

def query_lsh(
    lsh,
    dense_vectors,
    filenames,
    query_text,
    model,
    top_k=5,
    similarity_metric="euclidean"
):
    print("\n [STEP 1] Embedding query text...")
    t0 = time.time()

    query_vector = model.encode([query_text])[0]
    nonzero_features = np.count_nonzero(query_vector)

    print(f"  - Query vector shape: {query_vector.shape}")
    print(f"  - Non-zero embedding dimensions: {nonzero_features}")
    if nonzero_features == 0:
        print("Query vector is empty. Try another query.")
        return []

    print("\n [STEP 2] Querying LSH index...")
    candidate_ids = lsh.query(query_vector)
    t1 = time.time()

    print(f" - LSH returned {len(candidate_ids)} candidates in {t1 - t0:.3f} sec")

    if not candidate_ids:
        print(" No candidates found. Possibly due to hash size or vector sparsity.")
        return []

    print("\n [STEP 3] Matching candidate filenames to vectors...")
    candidate_indices = []
    for f in candidate_ids:
        try:
            idx = filenames.index(f)
            candidate_indices.append(idx)
        except ValueError:
            print(f" Candidate {f} not found in filenames list.")

    if not candidate_indices:
        print(" No valid candidate indices found.")
        return []

    candidate_vectors = dense_vectors[candidate_indices]
    print(f"  - Found {len(candidate_vectors)} vectors for comparison.")

    print("\n [STEP 4] Computing distances...")
    distances = []
    for i, vec in enumerate(tqdm(candidate_vectors, desc="Scoring")):
        if similarity_metric == "cosine":
            denom = np.linalg.norm(query_vector) * np.linalg.norm(vec) + 1e-8
            score = 1 - np.dot(query_vector, vec) / denom
        else:
            score = np.linalg.norm(query_vector - vec)
        distances.append((filenames[candidate_indices[i]], score))

    print("\n [STEP 5] Selecting top matches...")
    sorted_results = sorted(distances, key=lambda x: x[1])[:top_k]

    print(f"\n Top-{top_k} nearest results:")
    for name, score in sorted_results:
        print(f"  • {name} → Score: {score:.4f}")

    return sorted_results

In [18]:
query_text_mit = """
THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES 
OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE
LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
"""
query_text_no_license = """
The gentle rustling of leaves echoed through the forest as the sun dipped below the horizon, casting long shadows across the mossy
ground. Birds chirped their final songs of the day while the air grew cooler, carrying the scent of pine and damp earth. It was a
moment of calm, a brief pause in the world's endless motion.
"""

In [19]:
results = query_lsh(lsh, dense_vectors, sample_filenames, query_text_mit, model, top_k=5)


 [STEP 1] Embedding query text...
  - Query vector shape: (384,)
  - Non-zero embedding dimensions: 384

 [STEP 2] Querying LSH index...
[DEBUG] Table 0 hit → 1 candidates
[DEBUG] Table 3 hit → 2 candidates
[DEBUG] Table 4 hit → 2 candidates
[DEBUG] Table 6 hit → 1 candidates
[DEBUG] Table 7 hit → 1 candidates
[DEBUG] Table 8 hit → 3 candidates
[DEBUG] Table 9 hit → 1 candidates
[DEBUG] Table 11 hit → 1 candidates
[DEBUG] Table 12 hit → 1 candidates
[DEBUG] Table 13 hit → 2 candidates
[DEBUG] Table 15 hit → 2 candidates
[DEBUG] Table 18 hit → 1 candidates
[DEBUG] Table 19 hit → 1 candidates
[DEBUG] Table 22 hit → 2 candidates
[DEBUG] Table 25 hit → 1 candidates
[DEBUG] Table 28 hit → 3 candidates
[DEBUG] Table 29 hit → 1 candidates
 - LSH returned 4 candidates in 0.592 sec

 [STEP 3] Matching candidate filenames to vectors...
  - Found 4 vectors for comparison.

 [STEP 4] Computing distances...


Scoring: 100%|███████████████████████████████████| 4/4 [00:00<00:00, 681.59it/s]


 [STEP 5] Selecting top matches...

 Top-5 nearest results:
  • MITNFA/MITNFA-6.txt → Score: 0.0705
  • Imlib2/Imlib2-20.txt → Score: 0.1726
  • MITNFA/MITNFA-6_Split_SPDX_Licenses.txt → Score: 0.2865
  • Intel/Intel-26.txt → Score: 0.5118


In [20]:
results = query_lsh(lsh, dense_vectors, sample_filenames, query_text_no_license, model, top_k=5)


 [STEP 1] Embedding query text...
  - Query vector shape: (384,)
  - Non-zero embedding dimensions: 384

 [STEP 2] Querying LSH index...
 - LSH returned 0 candidates in 1.087 sec
 No candidates found. Possibly due to hash size or vector sparsity.


## Let's try on some more samples

In [13]:
import pandas as pd
df = pd.DataFrame(sample_filenames)

In [17]:
df

,0
0,OSL-2.1/OSL-2.1-37_Split_DB_Foss_Licenses.txt
1,OSL-2.1/OSL-2.1-117_Split_DB_Foss_Licenses.txt
2,OSL-2.1/OSL-2.1-141_Split_SPDX_Licenses.txt
3,OSL-2.1/OSL-2.1-206_Split_DB_Foss_Licenses.txt
4,OSL-2.1/OSL-2.1-65_Split_SPDX_Licenses.txt
...,...
9995,GFDL-1.2-only/GFDL-1.2-only-110.txt
9996,GFDL-1.2-only/GFDL-1.2-only-14.txt
9997,GFDL-1.2-only/GFDL-1.2-only-104.txt
9998,GFDL-1.2-only/GFDL-1.2-only-312.txt


In [26]:
unique_licenses = sorted({filename.split("/")[0] for filename in sample_filenames})

In [30]:
len(unique_licenses)

46

In [29]:
unique_licenses

['AAL',
 'BSD-3-Clause-Clear',
 'CATOSL',
 'CC-BY-3.0',
 'CC-BY-SA-3.0-AT',
 'CPL-0.5',
 'CrystalStacker',
 'FTL',
 'Ferris-1.2',
 'GFDL-1.1-no-invariants-or-later',
 'GFDL-1.2',
 'GFDL-1.2-invariants-or-later',
 'GFDL-1.2-only',
 'GFDL-1.3',
 'GFDL-1.3-invariants-or-later',
 'GPL-3.0-only',
 'GPL-3.0-with-bison-exception',
 'Imlib2',
 'Intel',
 'LiLiQ-P-1.1',
 'MITNFA',
 'MX4J',
 'NCGL-UK-2.0',
 'NLOD-1.0',
 'OASIS',
 'OLDAP-2.1',
 'OLDAP-2.6',
 'OLDAP-2.8',
 'OSL-2.1',
 'OpenMarket',
 'Parity-7.0.0',
 'PostgreSQL',
 'RealNetworks-EULA',
 'SCEA',
 'VIM',
 'W3C',
 'WashU',
 'YPL-1.0',
 'YPL-1.1',
 'ZPL-2.0',
 'ZPL-2.1',
 'Zed',
 'ZoneAlarm-EULA',
 'gnu-javamail-exception',
 'info-zip',
 'libtiff']

## Take samples for each license and non license too

In [33]:
import os
import random
from pathlib import Path

project_root = Path("..").resolve()
output_base = project_root / "Combined-Licenses"

def get_license_samples(base_dir):
    license_samples = []
    for subfolder in os.listdir(base_dir):
        full_path = os.path.join(base_dir, subfolder)
        if os.path.isdir(full_path):
            files = [f for f in os.listdir(full_path) if os.path.isfile(os.path.join(full_path, f))]
            if files:
                chosen_file = random.choice(files)
                file_path = os.path.join(full_path, chosen_file)
                with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
                    content = f.read().strip()
                    license_samples.append([content, 1, subfolder])  # Include folder name
    return license_samples

# Non-license texts: label = 0, folder name = None
non_license_texts = [
    "The quick brown fox jumps over the lazy dog.",
    "Artificial intelligence is transforming the world rapidly.",
    "He bought some apples and oranges from the supermarket.",
    "Python is a popular programming language for data science.",
    "The concert will be held at the downtown auditorium.",
    "Mountains are formed by tectonic plate movement.",
    "User authentication is essential for secure systems.",
    "She traveled to Europe last summer with her family.",
    "Electric vehicles are becoming more common in urban areas.",
    "Coffee consumption has increased during remote work periods.",
    "The novel was set in 19th-century London and Paris.",
    "Biology explores the diversity of life on Earth.",
    "He set up a local server for the development team.",
    "They planned to watch the meteor shower on Friday night.",
    "The history of computing dates back to the 1940s.",
    "Nutrition and exercise are crucial for a healthy life.",
    "The package will be delivered within 3-5 business days.",
    "Global warming poses a serious threat to ecosystems.",
    "The cat knocked over a vase while chasing a fly.",
    "Digital art has gained popularity through NFTs."
]
non_license_samples = [[text, 0, None] for text in non_license_texts]

license_samples = get_license_samples(output_base)
combined = license_samples + non_license_samples
random.shuffle(combined)


print(f"Total samples: {len(combined)} (License: {len(license_samples)}, Non-license: {len(non_license_samples)})")
for i, (text, label, folder) in enumerate(combined[:3]):
    print(f"\nSample {i+1} — Label: {label}, Folder: {folder}\n{text[:200]}...")  # Preview first 200 chars

Total samples: 674 (License: 654, Non-license: 20)

Sample 1 — Label: 1, Folder: Interbase
INTERBASE PUBLIC LICENSE Version 1.0 1 Definitions. 1.0.1 "Commercial Use" means distribution or otherwise making the Covered Code available to a third party. 1.1 ''Contributor'' means each entity tha...

Sample 2 — Label: 1, Folder: Glide
3DFX GLIDE Source Code General Public License 1 PREAMBLE This license is for software that provides a 3D graphics application program interface (API).The license is intended to offer terms similar to ...

Sample 3 — Label: 1, Folder: Font-exception-2.0
As a special exception, if you create a document which uses this font, and embed this font or unaltered portions of this font into the document, this font does not by itself cause the resulting docume...


In [34]:
def query_lsh_filenames_only(
    lsh,
    dense_vectors,
    filenames,
    query_text,
    model,
    top_k=5,
    similarity_metric="euclidean"
):
    results = query_lsh(
        lsh,
        dense_vectors,
        filenames,
        query_text,
        model,
        top_k=top_k,
        similarity_metric=similarity_metric
    )
    return [name for name, _ in results]

In [35]:
import pandas as pd

def evaluate_license_classification(dataset, lsh, dense_vectors, filenames, model):
    results = []

    for i, (text, label, folder) in enumerate(tqdm(dataset, desc="🔍 Evaluating Queries")):
        matched_files = query_lsh_filenames_only(
            lsh, dense_vectors, filenames, text, model, top_k=5
        )

        is_match = len(matched_files) > 0
        correct = (label == 1 and is_match) or (label == 0 and not is_match)

        results.append({
            "query_index": i,
            "text_preview": text[:100].replace("\n", " ") + "...",
            "true_label": label,
            "matched_files": matched_files,
            "correct": correct,
            "license_folder": folder
        })

    df_results = pd.DataFrame(results)
    accuracy = df_results["correct"].mean()
    print(f"\n Overall accuracy: {accuracy * 100:.2f}% ({df_results['correct'].sum()}/{len(df_results)})")
    return df_results


In [36]:
df_eval = evaluate_license_classification(combined, lsh, dense_vectors, sample_filenames, model)

🔍 Evaluating Queries:   0%|                            | 0/674 [00:00<?, ?it/s]


 [STEP 1] Embedding query text...


🔍 Evaluating Queries:   0%|                    | 1/674 [00:01<17:02,  1.52s/it]

  - Query vector shape: (384,)
  - Non-zero embedding dimensions: 384

 [STEP 2] Querying LSH index...
 - LSH returned 0 candidates in 1.519 sec
 No candidates found. Possibly due to hash size or vector sparsity.

 [STEP 1] Embedding query text...
  - Query vector shape: (384,)
  - Non-zero embedding dimensions: 384

 [STEP 2] Querying LSH index...
[DEBUG] Table 1 hit → 3 candidates
[DEBUG] Table 7 hit → 1 candidates
 - LSH returned 4 candidates in 0.062 sec

 [STEP 3] Matching candidate filenames to vectors...
  - Found 4 vectors for comparison.

 [STEP 4] Computing distances...



Scoring: 100%|███████████████████████████████████| 4/4 [00:00<00:00, 872.04it/s]

...

  - Query vector shape: (384,)
  - Non-zero embedding dimensions: 384

 [STEP 2] Querying LSH index...
 - LSH returned 0 candidates in 0.055 sec
 No candidates found. Possibly due to hash size or vector sparsity.

 [STEP 1] Embedding query text...
  - Query vector shape: (384,)
  - Non-zero embedding dimensions: 384

 [STEP 2] Querying LSH index...
 - LSH returned 0 candidates in 0.044 sec
 No candidates found. Possibly due to hash size or vector sparsity.

 [STEP 1] Embedding query text...
  - Query vector shape: (384,)
  - Non-zero embedding dimensions: 384

 [STEP 2] Querying LSH index...
 - LSH returned 0 candidates in 0.030 sec
 No candidates found. Possibly due to hash size or vector sparsity.

 [STEP 1] Embedding query text...


🔍 Evaluating Queries: 100%|█████████████████▉| 672/674 [01:29<00:00,  7.05it/s]

  - Query vector shape: (384,)
  - Non-zero embedding dimensions: 384

 [STEP 2] Querying LSH index...
 - LSH returned 0 candidates in 0.431 sec
 No candidates found. Possibly due to hash size or vector sparsity.

 [STEP 1] Embedding query text...
  - Query vector shape: (384,)
  - Non-zero embedding dimensions: 384

 [STEP 2] Querying LSH index...
 - LSH returned 0 candidates in 0.042 sec
 No candidates found. Possibly due to hash size or vector sparsity.

 [STEP 1] Embedding query text...
  - Query vector shape: (384,)
  - Non-zero embedding dimensions: 384

 [STEP 2] Querying LSH index...
 - LSH returned 0 candidates in 0.056 sec
 No candidates found. Possibly due to hash size or vector sparsity.

 [STEP 1] Embedding query text...
  - Query vector shape: (384,)
  - Non-zero embedding dimensions: 384

 [STEP 2] Querying LSH index...
[DEBUG] Table 1 hit → 2 candidates
[DEBUG] Table 5 hit → 42 candidates
[DEBUG] Table 9 hit → 42 candidates
[DEBUG] Table 13 hit → 40 candidates
[DEBUG] T


🔍 Evaluating Queries: 100%|██████████████████| 674/674 [01:29<00:00,  7.52it/s]


 [STEP 5] Selecting top matches...

 Top-5 nearest results:
  • OLDAP-2.6/OLDAP-2.6-27_Split_SPDX_Licenses.txt → Score: 0.2350
  • OLDAP-2.6/OLDAP-2.6-6.txt → Score: 0.2350
  • OLDAP-2.6/OLDAP-2.6-27.txt → Score: 0.2382
  • OLDAP-2.6/OLDAP-2.6-3_Split_SPDX_Licenses.txt → Score: 0.2382
  • OLDAP-2.6/OLDAP-2.6-25.txt → Score: 0.2382

✅ Overall accuracy: 36.94% (249/674)


In [39]:
df_eval

,query_index,text_preview,true_label,matched_files,correct,license_folder
0,0,INTERBASE PUBLIC LICENSE Version 1.0 1 Definit...,1,[],False,Interbase
1,1,3DFX GLIDE Source Code General Public License ...,1,"[CC-BY-3.0/CC-BY-3.0-371.txt, SCEA/SCEA-41_Spl...",True,Glide
2,2,"As a special exception, if you create a docume...",1,[gnu-javamail-exception/gnu-javamail-exception...,True,Font-exception-2.0
3,3,Lucent disclaims all warranties with regard to...,1,[],False,StandardML-NJ
4,4,\xe3\x82\xa2\xe3\x83\x88\xe3\x83\xaa\xe3\x83\x...,1,[],False,CC-BY-SA-2.1-JP
...,...,...,...,...,...,...
669,669,Common Documentation License Version 1.0 - Feb...,1,[],False,CDL-1.0
670,670,Sybase Open Watcom Public License version 1.0 ...,1,[],False,Watcom-1.0
671,671,Open CASCADE Technology Public License License...,1,[],False,OCCT
672,672,8. Indemnification...,1,[],False,Wintertree


In [43]:
# Filter only license-labeled samples
license_results = df_eval[df_eval['true_label'] == 1]

# Check which license folders were correctly identified
correctly_detected = license_results[license_results['correct'] == True]['license_folder'].unique()

# Cross-check against your list
correct_licenses = sorted(set(correctly_detected).intersection(set(unique_licenses)))
missing_licenses = sorted(set(unique_licenses) - set(correctly_detected))

print(f"\n Detected {len(correct_licenses)}/{len(unique_licenses)} target licenses correctly.\n")
print(" Correctly Detected Licenses:")
print("\n".join(correct_licenses))

print("\n Missing Licenses (not detected correctly):")
print("\n".join(missing_licenses))



 Detected 46/46 target licenses correctly.

🎯 Correctly Detected Licenses:
AAL
BSD-3-Clause-Clear
CATOSL
CC-BY-3.0
CC-BY-SA-3.0-AT
CPL-0.5
CrystalStacker
FTL
Ferris-1.2
GFDL-1.1-no-invariants-or-later
GFDL-1.2
GFDL-1.2-invariants-or-later
GFDL-1.2-only
GFDL-1.3
GFDL-1.3-invariants-or-later
GPL-3.0-only
GPL-3.0-with-bison-exception
Imlib2
Intel
LiLiQ-P-1.1
MITNFA
MX4J
NCGL-UK-2.0
NLOD-1.0
OASIS
OLDAP-2.1
OLDAP-2.6
OLDAP-2.8
OSL-2.1
OpenMarket
Parity-7.0.0
PostgreSQL
RealNetworks-EULA
SCEA
VIM
W3C
WashU
YPL-1.0
YPL-1.1
ZPL-2.0
ZPL-2.1
Zed
ZoneAlarm-EULA
gnu-javamail-exception
info-zip
libtiff

 Missing Licenses (not detected correctly):



In [45]:
non_license_results = df_eval[df_eval['true_label'] == 0]

correct_nonlicenses = non_license_results[non_license_results['correct'] == True]
incorrect_nonlicenses = non_license_results[non_license_results['correct'] == False]

print(f"\n Correctly identified {len(correct_nonlicenses)}/{len(non_license_results)} non-license texts.\n")

print(" Incorrectly identified these non-license texts as licenses:")
for i, row in incorrect_nonlicenses.iterrows():
    print(f" - Preview: {row['text_preview']}")
    print(f"   Matched files: {row['matched_files']}\n")


 Correctly identified 20/20 non-license texts.

 Incorrectly identified these non-license texts as licenses:


# Points to note:
- Identified all the licenses that were indexed correctly. (46 licenses in 10000 file samples)
- They were (46/654) licenses that were indexed into LSH.
- Identified all the non license text correclty. (20/674) samples.
- 608 licenses were not indexed, that is they were not part of the search space. Even then, it identified some statements that it had not seen previously. (203/608)
- Overall accuracy even with only (10000/162833) files indexed.

In [42]:
df_eval.to_csv("license_detection_results.csv", index=False)